# Prepare

## Exercise

The end result of this exercise should be a file named prepare.py.

## Using your store items data:



### 1. 
Convert date column to datetime format.


### 2. 
Plot the distribution of sale_amount and item_price.


### 3. 
Set the index to be the datetime variable.


### 4. 
Add a 'month' and 'day of week' column to your dataframe.


### 5. 
Add a column to your dataframe, sales_total, which is a derived from sale_amount (total items) and item_price.


### 6. 
Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions and be able to re-run the functions and get the same results.



## Using the OPS data acquired in the Acquire exercises opsd_germany_daily.csv, complete the following:



### 1. 
Convert date column to datetime format.


### 2. 
Plot the distribution of each of your variables.


### 3. 
Set the index to be the datetime variable.


### 4. 
Add a month and a year column to your dataframe.


### 5. 
Fill any missing values.


### 6. 
Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions and be able to re-run the functions and get the same results.

## Notes